In [5]:
#pip install langchain_community

In [4]:
# pip install llama-index-llms-ollama 

In [14]:
# pip install ipywidgets

In [1]:
# Import necessary libraries
import os
from pathlib import Path
import pandas as pd

# Import langchain modules
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain.document_loaders import UnstructuredExcelLoader, CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Import additional libraries
import urllib3
import json

http = urllib3.PoolManager()


## Run command in the terminal
```
docker exec -ti apan-ollama ollama pull gemma3:4b
```

In [17]:
# Function to fetch records from API and process them
def fetch_and_store_records():

    file_data = []
    if os.path.exists('summary.json'):
        with open('summary.json', 'r') as file:
            file_data = json.load(file)

    # Fetch records from API
    startPage = 11
    itemsPerPage = 2
    # response = http.request.get(f'http://apan-api:3100/api/v1/business/list?page={startPage}&items={itemsPerPage}&sortDir=ASC&sortBy=id')
    response = http.request('GET', (f'http://apan-api:3100/api/v1/business/list?page={startPage}&items={itemsPerPage}&sortDir=ASC&sortBy=id'))

    if response.status == 200:
            
        # Open json to save to resonse data
        data = json.loads(response.data)
        
        print(f"New API data: {len(data['result'])}")
        if len(data['result']) > 0:
            llm = Ollama(model="gemma3:4b", base_url="http://host.docker.internal:37869", verbose=True)

        for i, business in enumerate(data['result']):

            prompt = (
                f"I have a restaurant raw data as json like this {business}. Please generate business summary text like this 'This Roast Coffeehouse and Wine Bar fun place with takeout options. prices are inexpensive, catering available and it has an average 4-star review and etc.. so on'. Use all the available information for the summary text and Do not add any comments. Return final summary text only."
            )

            response = llm.invoke(prompt)

            print(f"response : {response}")
            # Append new response
            file_data.append({"business": business, "summary": response})

            # Write updated data back to the file
            with open('summary.json', 'w') as file:
                json.dump(file_data, file, indent=4)

    else:
        print(f"An error occurred: {response.status}")


# Call the function to fetch and store records
fetch_and_store_records()

New API data: 2
